In [1]:
import gzip
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import netCDF4 as nc
import match
from netCDF4 import Dataset
import xarray as xr
import os 
import pandas as pd  
from datetime import datetime

# 生成文件目录

In [3]:
start_year = 2016
end_year = 2021

folder_path = 'D:/data/SSH/2016-2021/'  # 文件夹路径
file_names = os.listdir(folder_path)  
len(file_names)  

2192

In [4]:
file_names[0],file_names[-1]

('dt_global_twosat_phy_l4_20160101_vDT2021.nc',
 'dt_global_twosat_phy_l4_20211231_vDT2021.nc')

In [5]:
# 打开所有nc文件
datasets = [xr.open_dataset(folder_path + file_name) for file_name in file_names]
# 合并多个数据集
merged_dataset = xr.concat(datasets, dim='time')  # 'time'是合并的维度

In [6]:
merged_dataset.to_netcdf('D:/data/SSH/SSH_2016-2021_concat.nc')

# 读取合并数据

In [7]:
nc = Dataset('D:/data/SSH/SSH_2016-2021_concat.nc')
print(nc.variables.keys())

for var in nc.variables.keys():
    data = nc.variables[var][:].data
    print(var,data.shape)

dict_keys(['crs', 'time', 'latitude', 'lat_bnds', 'longitude', 'lon_bnds', 'nv', 'sla', 'err_sla', 'ugosa', 'err_ugosa', 'vgosa', 'err_vgosa', 'adt', 'ugos', 'vgos', 'tpa_correction', 'flag_ice'])
crs (2192,)
time (2192,)
latitude (720,)
lat_bnds (2192, 720, 2)
longitude (1440,)
lon_bnds (2192, 1440, 2)
nv (2,)
sla (2192, 720, 1440)
err_sla (2192, 720, 1440)
ugosa (2192, 720, 1440)
err_ugosa (2192, 720, 1440)
vgosa (2192, 720, 1440)
err_vgosa (2192, 720, 1440)
adt (2192, 720, 1440)
ugos (2192, 720, 1440)
vgos (2192, 720, 1440)
tpa_correction (2192,)
flag_ice (2192, 720, 1440)


In [8]:
adt = nc.variables['adt'][:].data
sla = nc.variables['sla'][:].data
adt_mask =  nc.variables['adt'][:].mask
sla_mask =  nc.variables['sla'][:].mask

crs   =  nc.variables['crs'][:].data  #this is a container variable that describes the gird_mapping used by the data in this file.This variable does not contain any data;only information about the geographic coordinate system

lat   =  nc.variables['latitude'][:].data
lon   =  nc.variables['longitude'][:].data

In [9]:
adt[adt == -2147483647] = np.nan
sla[sla == -2147483647] = np.nan

In [10]:
start_year = 2016
end_year = 2021
time = []
date_range = pd.date_range(datetime(start_year,1,1),datetime(end_year,12,31),freq='1d')
len(date_range)
for i in range(len(date_range)):
    time.append(i)

In [11]:
len(time)

2192

# 保存数据

In [12]:
crs[0]

-2147483647

In [15]:
new_NC.close()

In [14]:
#new_NC.close()
new_NC = Dataset("D:/data/SSH/SSH_2016-2021_day.nc", 'w', format='NETCDF4')

new_NC.createDimension('lat', len(lat))
new_NC.createDimension('lon', len(lon))
new_NC.createDimension('time', len(time))
new_NC.createDimension('crs', 1)

#定义变量，这里需要规定变量的类型，以及限制它的维度
#可以看到，四个与数据相关的变量，其由另外三个基本维度约束

time_var = new_NC.createVariable('time', 'f4',("time"))
time_var.units = 'days since 2016-01-01'
time_var.long_name = 'days'
time_var.axis = 'T' 

new_NC.createVariable('lat', 'f', ("lat"))
new_NC.createVariable('lon', 'f', ("lon"))

new_NC.createVariable('ssh', 'f',("time","lat","lon"))
new_NC.createVariable('sla', 'f',("time","lat","lon"))
new_NC.createVariable('ssh_mask', 'f',("time","lat","lon"))
new_NC.createVariable('sla_mask', 'f',("time","lat","lon"))

new_NC.createVariable('crs', 'f',("crs"))


#向变量中填充数据
new_NC.variables['lat'][:] = lat
new_NC.variables['lon'][:] = lon
new_NC.variables['time'][:] = np.array(time)

new_NC.variables['ssh'][:]=np.array(adt)
new_NC.variables['sla'][:]=np.array(sla)
new_NC.variables['ssh_mask'][:]=np.array(adt_mask)
new_NC.variables['sla_mask'][:]=np.array(sla_mask)

new_NC.variables['crs'][:]=np.array(crs[0])

#最后记得关闭文件
new_NC.close()



# 按月求平均值

In [19]:
#adt.shape

(1826, 720, 1440)

In [15]:
import numpy as np
import calendar

# 假设data是您的数据，形状为 (4017, 720, 1440)
# 获取数据的形状
#1826       720        1440
num_days, num_rows, num_cols = adt.shape

# 创建一个数组来存储月平均值
monthly_averages_adt = []
monthly_averages_sla = []
start_day = 0 

for year in range(2016, 2022):  # 从2016年到2021年
    for month in range(1, 13):  # 1月到12月
        # 获取当前月份的天数，考虑年份和闰年
        days_in_month = calendar.monthrange(year, month)[1]
        # 在数据中选择当前月份的数据
        end_day = start_day + days_in_month
#         print(year,'年',month,'月')
#         print(start_day,end_day)
        monthly_data_adt = adt[start_day:end_day]
        monthly_data_sla = sla[start_day:end_day]
        # 计算每个月的平均值，并添加到结果数组中
        monthly_average_adt = np.nanmean(monthly_data_adt, axis=0)
        monthly_average_sla = np.nanmean(monthly_data_sla, axis=0)
        
        monthly_averages_adt.append(monthly_average_adt)
        monthly_averages_sla.append(monthly_average_sla)
        
        start_day = end_day

C:\Users\Administrator\AppData\Local\Temp\ipykernel_69608\2539997583.py:25: RuntimeWarning: Mean of empty slice
  monthly_average_adt = np.nanmean(monthly_data_adt, axis=0)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_69608\2539997583.py:26: RuntimeWarning: Mean of empty slice
  monthly_average_sla = np.nanmean(monthly_data_sla, axis=0)


In [16]:
len(monthly_averages_adt)

72

## 获取时间

In [17]:
start_year = 2016
end_year =2021
time = []
date_range = pd.date_range(datetime(start_year,1,1),datetime(end_year,12,31),freq='1m')
len(date_range)
for i in range(len(date_range)):
    time.append(i)

In [18]:
len(time)

72

# 保存月平均数据

In [19]:
#new_NC.close()
new_NC = Dataset("D:/data/SSH/SSH_2016-2021_month.nc", 'w', format='NETCDF4')

'''
定义维度，后一个参数表示维度的长度，因为是合并的同一个产品的数据，所以是统一
的,注意维度的长度一定要和读入的数据匹配
'''
new_NC.createDimension('lat', len(lat))
new_NC.createDimension('lon', len(lon))
new_NC.createDimension('time', len(time))
new_NC.createDimension('crs', 1)



time_var = new_NC.createVariable('time', 'f4',("time"))
time_var.units = 'months since 2016-01-15'
time_var.long_name = 'months'
time_var.axis = 'T' 


new_NC.createVariable('lat', 'f', ("lat"))
new_NC.createVariable('lon', 'f', ("lon"))

new_NC.createVariable('ssh', 'f',("time","lat","lon"))
new_NC.createVariable('sla', 'f',("time","lat","lon"))
new_NC.createVariable('ssh_mask', 'f',("time","lat","lon"))
new_NC.createVariable('sla_mask', 'f',("time","lat","lon"))

new_NC.createVariable('crs', 'f',("crs"))


#向变量中填充数据
new_NC.variables['lat'][:] = lat
new_NC.variables['lon'][:] = lon
new_NC.variables['time'][:] = np.array(time)

new_NC.variables['ssh'][:]=np.array(monthly_averages_adt)
new_NC.variables['sla'][:]=np.array(monthly_averages_sla)
new_NC.variables['ssh_mask'][:]=np.array(adt_mask[0:1])
new_NC.variables['sla_mask'][:]=np.array(sla_mask[0:1])

new_NC.variables['crs'][:]=np.array(crs[0])

#最后记得关闭文件
new_NC.close()



In [34]:
adt_mask[0:1].shape

(1, 720, 1440)